In [2]:
import pandas as pd

In [3]:
df = pd.read_csv ('../data/michelin_reviews.csv')

In [4]:
df

,Unnamed: 0,reviews,Unnamed: 2
0,0,\nChef Barry Dindyal is the talent behind this...,NaN
1,1,\nIt may be the signature cry for falling obje...,NaN
2,2,\nOnce a pop-up and now a full-fledged brick-a...,NaN
3,3,\nChef Rob Sonderman of the Federalist Pig has...,NaN
4,4,\nThis cozy Ethiopian den is an homage to Owne...,NaN
...,...,...,...
1375,1375,\nThe architecturally intriguing Hayden Tract ...,NaN
1376,1376,"\nSet amidst the luxury shops of Rodeo Drive, ...",NaN
1377,1377,"\nIn contrast to its grand hotel setting, this...",NaN
1378,1378,"\nEvery quarter of the year, Maude finds its n...",NaN


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(max_features=10_000,stop_words="english")
features = vec.fit_transform(df["reviews"])

print(features.shape) # (11314, 10000)

(1380, 10000)


In [6]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2, metric='cosine',n_jobs=-1)
knn.fit(features)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=2)

In [7]:
philly_df = pd.read_csv ('../data/philly_review_p.csv')

In [8]:
philly_urls = pd.read_csv('../data/philly_infatuation_review_urls.csv')
philly_urls

,/philadelphia/reviews/spice-finch
0,/philadelphia/reviews/bookers-restaurant-bar
1,/philadelphia/reviews/jims-south-st
2,/philadelphia/reviews/fork
3,/philadelphia/reviews/laser-wolf
4,/philadelphia/reviews/fiorella-1
...,...
569,/philadelphia/reviews/burgs-hideaway-lounge
570,/philadelphia/reviews/pumpkin-restaurant
571,/philadelphia/reviews/day-by-day
572,/philadelphia/reviews/sams-morning-glory-diner


In [9]:
merged_df = philly_df.merge(philly_urls, left_index=True, right_index=True)

In [10]:
input_texts = merged_df['reviews']
input_features = vec.transform(input_texts)

distance = []
neighbor = []

D, N = knn.kneighbors(input_features, n_neighbors=1, return_distance=True)

for input_text, distances, neighbors in zip(input_texts, D, N):
    for dist, neighbor_idx in zip(distances, neighbors):
        distance.append(dist)
        neighbor.append(neighbor_idx)

In [11]:
distance

[0.8603233771733889,
 0.8848072785509556,
 0.868340134348899,
 0.8979495852825917,
 0.8857811691712298,
 0.8392495305056988,
 0.9051395629949104,
 0.8570249558013574,
 0.8902656604902416,
 0.8587667552379106,
 0.8662955538625038,
 0.851480095441276,
 0.8422238598244465,
 0.8661939290762126,
 0.829139153311466,
 0.906040424546635,
 0.8780924114916717,
 0.8656284218051614,
 0.8862460892504432,
 0.8603130736778211,
 0.8977054891271194,
 0.8539116699039986,
 0.8775863271383031,
 0.8001976369661483,
 0.9244635789109907,
 0.8877049187267694,
 0.9070263418630194,
 0.7904144773706709,
 0.9024485021946221,
 0.8600457868351762,
 0.8766335299708851,
 0.8050525739676907,
 0.8787156315453404,
 0.8373945859110942,
 0.8755645831493516,
 0.8870170795982026,
 0.814731015322379,
 0.8568990433605165,
 0.841881131552128,
 0.825660093829615,
 0.8399348112309928,
 0.8869773956820927,
 0.8954283218435899,
 0.9002608624889378,
 0.7793217092943021,
 0.8848311764982728,
 0.9104323171906341,
 0.908146027826108,


In [12]:
merged_df['distance'] = distance

In [13]:
merged_df['neighbor'] = neighbor

In [16]:
merged_df

,reviews,/philadelphia/reviews/spice-finch,distance,neighbor
0,"Most of the time, it’s fun to be right. It’s f...",/philadelphia/reviews/bookers-restaurant-bar,0.860323,290
1,When we’re in the mood for a creamy bowl of gr...,/philadelphia/reviews/jims-south-st,0.884807,285
2,"Jim’s is a retro, counter-service place on Sou...",/philadelphia/reviews/fork,0.868340,1298
3,You like Fork. Your boss likes Fork. Your neig...,/philadelphia/reviews/laser-wolf,0.897950,517
4,Our review for this Israeli grill could probab...,/philadelphia/reviews/fiorella-1,0.885781,880
...,...,...,...,...
569,Burg’s Hideaway is a small bar in Point Breeze...,/philadelphia/reviews/burgs-hideaway-lounge,0.882979,1311
570,If your date’s the type that does all of their...,/philadelphia/reviews/pumpkin-restaurant,0.906682,1013
571,"No matter the weekend or the weather, Day by D...",/philadelphia/reviews/day-by-day,0.851864,422
572,There are mornings you feel invincible. You kn...,/philadelphia/reviews/sams-morning-glory-diner,0.904479,1213


In [ ]:
pd.write_cs

In [15]:
merged_df.loc[109]

reviews                              Just like how Nicolas Cage’s career is the sum...
/philadelphia/reviews/spice-finch                         /philadelphia/reviews/la-peg
distance                                                                      0.864942
neighbor                                                                           180
Name: 109, dtype: object